In [ ]:
import cv2

import numpy as np
from scipy import ndimage as ndi
import geopandas as gpd
from skimage.morphology import watershed
from skimage.feature import peak_local_max
import matplotlib.pyplot as plt
import rasterio
from shapely.geometry import Point, LineString, Polygon
from rasterio import features
from rasterio import Affine
from shapely.affinity import scale as shapely_scale


%matplotlib inline

In [ ]:
img_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/angle_training_180/504/edges/0_0_1969_0_1969.png"

In [ ]:
raster_mask = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/v5_tta_full_504/rgb_504_rgg_bin.tif"
rows_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/row_data/504/row_data.geojson"
detected_polygon_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/row_data/504/detected_polygon.geojson"

In [ ]:
row_df = gpd.read_file(rows_path)
dataset = rasterio.open(raster_mask, "r")

In [ ]:
dataset.height

In [ ]:
def lines_to_image_view(points, dataset, downscale=None, rect=None):
    if rect:
        col_offset = rect[0]
        row_offset = rect[1]
    img_polyg_array = []
    if downscale:
        to_img_mat = ~dataset.meta['transform']
        #to_img_mat = ~dataset.meta['transform']
    else:
        to_img_mat = ~dataset.meta['transform']
    for item in points.iterrows():
        polyg_spati = np.array(item[1]['geometry'].coords)
        polyg_img = [np.array(tuple(pt) * to_img_mat) / downscale for pt in polyg_spati]
        if rect:
            polyg_img = np.subtract(polyg_img, (col_offset, row_offset))
        polyg_img = LineString(polyg_img)
        # polyg_img = polyg_img.convex_hull
        img_polyg_array.append(polyg_img)

    return img_polyg_array

In [ ]:
~dataset.meta['transform']

In [ ]:
rows_img = lines_to_image_view(row_df, dataset, downscale = 6.5, rect=None)

In [ ]:
mid_line = features.rasterize(rows_img, out_shape=(int(dataset.width /  6.5), int(dataset.height /  6.5)))
kernel = np.ones((10, 10))
dilated_mid_line = cv2.dilate(mid_line, kernel)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(dilated_mid_line)

In [ ]:
plt.imshow(mid_line)

In [ ]:
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

In [ ]:
image = dilated_mid_line

In [ ]:
mask_x = np.ones(image.shape) * 255

In [ ]:
image.shape[0] / 300

In [ ]:
plt.imshow(image)

In [ ]:
# Now we want to separate the two objects in image
# Generate the markers as local maxima of the distance to the background
distance = ndi.distance_transform_edt(image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)),
                            labels=mask_x)
markers = ndi.label(image)[0]
labels = watershed(-distance, markers, mask=mask_x)
plt.figure(figsize=(20,20))
fig, axes = plt.subplots(ncols=3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Overlapping objects')
ax[1].imshow(-distance, cmap=plt.cm.gray)
ax[1].set_title('Distances')
ax[2].imshow(labels, cmap=plt.cm.nipy_spectral)
ax[2].set_title('Separated objects')

for a in ax:
    a.set_axis_off()

fig.tight_layout()

In [ ]:
np.unique(labels)

In [ ]:
polys_shapely = []
for point in np.unique(labels):
    tmp = labels.copy()
    tmp[np.where(labels != point)] = 0
    tmp = (tmp > 0).astype("uint8")
    _, contour, _ = cv2.findContours(tmp, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #contours.append(contour)
    c_points = [pt for pt in contour[0].squeeze()]
    try:
        polygon = shapely_scale(Polygon(c_points), 6.5, 6.5, origin=(0,0,0))
        polyg_geo = [np.array(tuple(pt) * dataset.meta['transform']) for pt in polygon.exterior.coords]
        polyg_geo = Polygon(polyg_geo)
        polys_shapely.append(polyg_geo)
    except:
        raise

In [ ]:
import os

detected_polygon_path = "/mnt/storage_4tb/ymi/geo_data/40m_for_ori/citrusuco/angle_training_180/504/line_polys.geojson"

polygon_df = gpd.GeoDataFrame(geometry=polys_shapely)
polygon_df.crs = dataset.meta.get('crs')
if os.path.exists(detected_polygon_path):
    os.remove(detected_polygon_path)
polygon_df.to_file(detected_polygon_path, driver='GeoJSON', encoding='utf-8')

In [ ]:
plt.figure(figsize=(20,20))

plt.imshow(labels, cmap='flag')

In [ ]:
labels